In [2]:
import warnings
warnings.filterwarnings("ignore")

%pylab
import LearnModCoupled as LM
import random
import sys
import pickle

from scipy.integrate import odeint
from math import factorial
from numpy.random import choice
import numba
from IPython.display import clear_output

NN = 64
NNet = 1
RandSeed = 1
net, net2 = LM.LoadNetwork(NN, NNet, RandSeed)

#setup useful network objects
DIM = 1
#NN = net2.NN
NE = net2.NE
EI = array(net2.edgei)
EJ = array(net2.edgej)

Conv = zeros([NE, NN])
for i in range(NE):
    Conv[i,EI[i]] = +1.
    Conv[i,EJ[i]] = -1.

# setup matrices
D1 = array([where(EI==i,1,0) for i in range(NN)])
D2 = array([where(EJ==j,1,0) for j in range(NN)])
DD = D1 + D2

DM = zeros([NE, NN])
for i in range(NE):
    DM[i,EI[i]] = 1
    DM[i,EJ[i]] = -1

ids = c_[EI,EJ]
UT = zeros([NN,NN])
G = zeros(NN); G[-1] = 1.


Trials = 1
numT2s = 1
Iters = 30000

AllCosts = zeros([Trials, numT2s, Iters+1])
AllCostsNEq = zeros([Trials, numT2s, Iters+1])
EqDistances = zeros([Trials, numT2s, Iters])
DotProds = zeros([Trials, numT2s, Iters])
TrainedSteps = ones([Trials, numT2s]) * nan
TrainedRT = ones([Trials, numT2s]) * nan
Taus = zeros([Trials])
AllCostsCL = zeros([Trials, Iters+1])
TrainedStepsCL = ones([Trials]) * nan

Thresh0 = 0.05
showEvery = 1000

import jax.numpy as jnp
import jax.numpy.linalg as jla
from jax.ops import index_update, index
#from jax.scipy.linalg import solve as jsolve
from jax.numpy.linalg import solve as jsolve
from jax import grad, jit, vmap

def JGetPs(Data, Nodes, K, DD, UT, G):
    NN = len(DD)
    D = jnp.diag(jnp.dot(DD, K))
    
    #UT = jnp.zeros([NN,NN])
    #UT[EI,EJ] = -K
    UT = index_update(UT, index[EI,EJ], -K)
    LD = D + UT + UT.T

    cs = len(Nodes)
    id2 = jnp.arange(cs)

    S = jnp.zeros([NN, cs])
    S = index_update(S, index[Nodes,id2], +1.)

    LDB = jnp.zeros([NN+1+cs, NN+1+cs])
    LDB = index_update(LDB, index[:NN,:NN], LD)
    LDB = index_update(LDB, index[NN,:NN], G)
    LDB = index_update(LDB, index[:NN,NN], G)
    LDB = index_update(LDB, index[:NN,NN+1:], S)
    LDB = index_update(LDB, index[NN+1:,:NN], S.T)
    
    f = jnp.zeros(NN+1+cs)
#     f[NN+1:] = Data
    f = index_update(f, index[NN+1:], Data)
    
    P = jsolve(LDB, f)[:NN]
    return P

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
alpha = 0.1
eta   = 1.e-3
sources = 5
targets = 3
numT2s = 19

sources = 5
targets = 20
RNodes = choice(arange(NN-1), size=sources+targets, replace=False)
KNodes = list(set(arange(NN)) - set(RNodes))
SourceNodes = RNodes[:sources]
TargetNodes = RNodes[sources:]        
HiddenNodes = list(set(arange(NN)) - set(SourceNodes) - set(TargetNodes))
ConsNodes = r_[SourceNodes, TargetNodes]
OtherNodes = list(set(arange(NN)) - set(SourceNodes) - set(TargetNodes))
Data = randn(1,sources)  # inputs
Comp = randn(targets) * 0.2
STNodes = r_[SourceNodes, TargetNodes]
NSNodes = array(list(set(range(NN)) - set(SourceNodes)))
NSTNodes = array(list(set(range(NN)) - set(STNodes)))


Iters = 50000

# setup matrices
D1 = array([where(EI==i,1,0) for i in range(NN)])
D2 = array([where(EJ==j,1,0) for j in range(NN)])
DD = D1 + D2

ids = c_[EI,EJ]
UT = zeros([NN,NN])
G = zeros(NN); G[-1] = 1.


AllCosts = zeros([Trials, numT2s, Iters+1])
AllCostsNEq = zeros([Trials, numT2s, Iters+1])
EqDistances = zeros([Trials, numT2s, Iters])
DotProds = zeros([Trials, numT2s, Iters])
TrainedSteps = ones([Trials, numT2s]) * nan
TrainedRT = ones([Trials, numT2s]) * nan
Taus = zeros([Trials])
AllCostsCL = zeros([Trials, Iters+1])
TrainedStepsCL = ones([Trials]) * nan

Thresh0 = 0.01
showEvery = 2000


K0 = ones(NE)
DiagK0 = diag(K0)
PF0 = LM.GetPs(Data, SourceNodes, K0, NN, NE, EI, EJ, DD, UT, G)
DPF0 = PF0[EI] - PF0[EJ]

#Comp = PF0[TargetNodes] + 0.001*randn(targets)

Nudge = eta * (Comp - PF0[TargetNodes])
PC0 = LM.GetPs(r_[Data[0], PF0[TargetNodes] + Nudge], r_[SourceNodes,TargetNodes], K0, NN, NE, EI, EJ, DD, UT, G)
DPC0 = PF0[EI] - PF0[EJ]


Params = [NN, NE, EI, EJ, SourceNodes, TargetNodes, STNodes, NSNodes, NSTNodes, Comp, DM, eta, alpha]

Y0 = r_[PF0, PC0, K0]

In [5]:
from scipy.integrate import solve_ivp
eta   = 1.e-3
alpha = 0.1
eta   = 1.e-3
Ratios = logspace(-12.,3.,46)
Thresh = 1.e-3

# setup matrices
D1 = array([where(EI==i,1,0) for i in range(NN)])
D2 = array([where(EJ==j,1,0) for j in range(NN)])
DD = D1 + D2

DM = zeros([NE, NN])
for i in range(NE):
    DM[i,EI[i]] = 1
    DM[i,EJ[i]] = -1

ids = c_[EI,EJ]
UT = zeros([NN,NN])
G = zeros(NN); G[-1] = 1.

sources = 5
targets = 3
RNodes = choice(arange(NN-1), size=sources+targets, replace=False)
KNodes = list(set(arange(NN)) - set(RNodes))
SourceNodes = RNodes[:sources]
TargetNodes = RNodes[sources:]        
HiddenNodes = list(set(arange(NN)) - set(SourceNodes) - set(TargetNodes))
ConsNodes = r_[SourceNodes, TargetNodes]
OtherNodes = list(set(arange(NN)) - set(SourceNodes) - set(TargetNodes))
Data = randn(1,sources)  # inputs
Comp = randn(targets) * 0.2 # outputs
STNodes = r_[SourceNodes, TargetNodes]
NSNodes = array(list(set(range(NN)) - set(SourceNodes)))
NSTNodes = array(list(set(range(NN)) - set(STNodes)))

PeakDiff = zeros(len(Ratios))
EqDists = zeros(len(Ratios))
IntDots = zeros(len(Ratios))
Ts = []
CEqs = []
CNEqs = []
CE2 = []
CNE2 = []
AEDists = []
ADots = []
ADK = []
ADKCL = []
ADPF = []
ADPC = []
AKC = []
AKEnd = []
TrainTime = []

for i in range(len(Ratios)):
    gamma = 1.
    alpha = gamma / Ratios[i]
    K0 = ones(NE)
    PF0 = LM.GetPs(Data, SourceNodes, K0, NN, NE, EI, EJ, DD, UT, G)
    DPF0 = PF0[EI] - PF0[EJ]
    PPF0 = K0 * DPF0**2
    
    Nudge = eta * (Comp - PF0[TargetNodes])
    PC0 = LM.GetPs(r_[Data[0], PF0[TargetNodes] + Nudge], r_[SourceNodes,TargetNodes], K0, NN, NE, EI, EJ, DD, UT, G)
    DPC0 = PC0[EI] - PC0[EJ]
    PPC0 = K0 * DPC0**2
    Y0 = r_[PF0, PC0, K0]
    #Y0 = r_[PF0, PF0, K0]

    C0 = sum((PF0[TargetNodes] - Comp)**2.)
    Event = LM.LearningCompleteEq
    Params = [NN, NE, EI, EJ, SourceNodes, TargetNodes, STNodes, NSNodes, NSTNodes, Comp, DM, eta, alpha, gamma, C0, Thresh, DD, UT, G]

    TIntegrate = 1.e8
    if alpha > 1:
        Res = solve_ivp(LM.dPKdtSwitch, (0., TIntegrate), Y0, args=(Params,), method='RK45', events=Event)
    else:
        Res = solve_ivp(LM.dPKdtSwitch, (0., TIntegrate), Y0, args=(Params,), method='LSODA', events=Event)
    sol = Res.y.T
    t = Res.t

    PFs = sol[:,:NN]
    PCs = sol[:,NN:2*NN]
    Ks = sol[:,2*NN:]
    PFsEq = array([LM.GetPs(Data, SourceNodes, Ks[i], NN, NE, EI, EJ, DD, UT, G) for i in range(len(PFs))])
    #PCsEq = array([LM.GetPs(r_[ Data[0], PFsEq[i][TargetNodes] + eta * (Comp - PFsEq[i][TargetNodes]) ], r_[SourceNodes, TargetNodes], Ks[i], NN, NE, EI, EJ, DD, UT, G) for i in range(len(PFs))])

    EqDist = array([norm(PFs[i] - PFsEq[i]) for i in range(0, len(PFs)) ])
    #EqDist = array([norm(PFs[i] - PFsEq[i])/norm(PCs[i] - PFs[i]) for i in range(0, len(PFs)) ])
    EDM = EqDist.max()
    
    IED = trapz(EqDist, t)/t[-1]
    
    DPFs = array([PFs[i][EI] - PFs[i][EJ] for i in range(len(PFs))])
    PCs = sol[:,NN:2*NN]
    PCs[:, TargetNodes] = array([(1. - eta) * PFs[i, TargetNodes] + eta * Comp for i in range(len(PCs))])
    DPCs = array([PCs[i][EI] - PCs[i][EJ] for i in range(len(PCs))])
    DKs = alpha/eta * (DPFs**2 - DPCs**2)
    NDKs = array([DKs[i]/norm(DKs[i]) for i in range(len(DKs))])
    PFsEq = array([LM.GetPs(Data, SourceNodes, Ks[i], NN, NE, EI, EJ, DD, UT, G) for i in range(len(PFs))])
    DPFsEq = array([PFsEq[i][EI] - PFsEq[i][EJ] for i in range(len(PFsEq))])
    PCsEq = array([LM.GetPs(r_[ Data[0], PFsEq[i][TargetNodes] + eta * (Comp - PFsEq[i][TargetNodes]) ], r_[SourceNodes, TargetNodes], Ks[i], NN, NE, EI, EJ, DD, UT, G) for i in range(len(Ks))])
    DPCsEq = array([PCsEq[i][EI] - PCsEq[i][EJ] for i in range(len(PCsEq))])
    DKsEq = alpha/eta * (DPFsEq**2 - DPCsEq**2)
    NDKsEq = array([DKsEq[i]/norm(DKsEq[i]) for i in range(len(DKsEq))])
    Dots = array([dot(NDKs[i], NDKsEq[i]) for i in range(len(NDKs))])
    CThreshTrapz = 1.e-4
    CostEq = sum((PFsEq[:,TargetNodes] - Comp)**2., axis=1)
    CostNEq = sum((PFs[:,TargetNodes] - Comp)**2., axis=1)
    C = CostEq/CostEq[0]
    TIND = -1
    if len(where(C < CThreshTrapz)[0]) > 0 :
        TIND = where(C < CThreshTrapz)[0][0]
    INTD = trapz(Dots[:TIND], t[:TIND])/t[TIND-1]
    
    PPFsEq = Ks * DPFsEq**2
    PPCsEq = Ks * DPCsEq**2
    CostE2 = sum(PPCsEq - PPFsEq, 1) / eta
    
    PPFs = Ks * DPFs**2
    PPCs = Ks * DPCs**2
    CostNE2 = sum(PPCs - PPFs, 1) / eta
    
    PeakDiff[i] = EDM
    EqDists[i] = IED
    IntDots[i] = INTD
   
    
    print(i, Ratios[i], EDM, IED, 1.-INTD, C[-1], t[-1])
    
    Ts.append(t)
    CEqs.append(CostEq)
    CNEqs.append(CostNEq)
    CE2.append(CostE2)
    CNE2.append(CostNE2)
    AEDists.append(EqDist)
    ADots.append(Dots)
    ADK.append(norm(DKs,axis=1))
    ADKCL.append(norm(DKsEq,axis=1))
    ADPF.append(norm(PFs[1:,:]-PFs[:-1,:], axis=1)/diff(t))
    ADPC.append(norm(PCs[1:,:]-PCs[:-1,:], axis=1)/diff(t))
    TrainTime.append(t[-1])
    AKC.append(norm(Ks[-1] - K0))
    AKEnd.append(Ks[-1])
    
    Save = {}
    Save['NN'] = NN
    Save['NE'] = NE
    Save['EI'] = EI
    Save['EJ'] = EJ
    Save['NetNumber'] = NNet
    Save['RandSeed'] = RandSeed
    Save['sources'] = sources
    Save['targets'] = targets
    Save['Data'] = Data
    Save['Comp'] = Comp
    Save['SourceNodes'] = SourceNodes
    Save['TargetNodes'] = TargetNodes
    Save['eta'] = eta
    Save['alpha'] = alpha
    Save['Iters'] = Iters
    Save['TIntegrate'] = TIntegrate
    Save['Thresh'] = Thresh
    Save['Ratios'] = Ratios
    Save['Ts'] = Ts
    Save['CEqs'] = CEqs
    Save['CNEqs'] = CNEqs
    Save['CE2'] = CE2
    Save['CNE2'] = CNE2
    Save['AEDists'] = AEDists
    Save['ADots'] = ADots
    Save['ADK'] = ADK
    Save['ADKCL'] = ADKCL
    Save['ADPF'] = ADPF
    Save['ADPC'] = ADPC
    Save['PeakDiff'] = PeakDiff
    Save['EqDists'] = EqDists
    Save['IntDots'] = IntDots
    Save['TrainTime'] = TrainTime
    Save['AKC'] = AKC
    Save['AKEnd'] = AKEnd

    import pickle
    fn = 'NonEqData.pkl'
    f = open(fn, 'wb')
    pickle.dump(Save, f)
    f.close()

0 1e-12 4.800251581578712 1.4561219050978096 1.0817863354141166 0.0009999999999999879 0.00012856662450861072
1 2.1544346900318868e-12 4.796099502543065 1.454623201165757 1.080154812766772 0.000999999999999963 0.00018892372197627338
2 4.6415888336127725e-12 4.790370319546658 1.455093474417514 1.0808632205251403 0.0009999999999963651 0.0002771717957830958
3 1e-11 6.249852563627824 1.4673458065799585 1.0806796269001167 0.0010000000000000423 0.0004073306684703303
4 2.1544346900318868e-11 4.765782313800076 1.4539236139272238 1.0822934942046047 0.0009999999999999707 0.0005979559866351097
5 4.641588833612773e-11 4.753733267593215 1.453472048175925 1.082442227964512 0.0009999999999999736 0.000876322420561839
6 1e-10 4.733047617109656 1.4527420419784565 1.0837592666884774 0.0009999999999999976 0.0012870891616957119
7 2.1544346900318779e-10 4.708517822214947 1.4519321171122357 1.083620416299547 0.00099999999999992 0.0018889222309213319
8 4.6415888336127727e-10 4.668468475304778 1.447335318530092